In [71]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import joblib
import streamlit as st

In [72]:
# 폰트지정
plt.rcParams['font.family'] = 'Malgun Gothic'




# 마이너스 부호 깨짐 지정
plt.rcParams['axes.unicode_minus'] = False




# 숫자가 지수표현식으로 나올 때 지정
pd.options.display.float_format = '{:.2f}'.format

In [73]:
# 1. 데이터 로드 및 전처리
train_df = pd.read_csv("dataset/merged_cpi_dataset_test.csv")
test_df = pd.read_csv("dataset/merged_cpi_dataset_train.csv")

In [74]:
# 데이터 확인
train_df.head()

,날짜,CPI,금리,환율
0,2000-01-01,1.90,9.28,1123.20
1,2000-02-01,1.80,8.99,1131.00
2,2000-03-01,2.10,9.06,1106.00
3,2000-04-01,1.40,8.95,1109.10
4,2000-05-01,1.10,8.96,1129.40


In [75]:
# 결측치 확인
print("Train 데이터 결측치 확인:")
print(train_df.isnull().sum())
print("\nTest 데이터 결측치 확인:")
print(test_df.isnull().sum())

Train 데이터 결측치 확인:
날짜     0
CPI    0
금리     0
환율     0
dtype: int64

Test 데이터 결측치 확인:
날짜     0
CPI    0
금리     0
환율     0
dtype: int64


In [76]:
# 불필요한 컬럼 제거 (원핫 인코딩 없이도 잘 작동하도록)
train_df = train_df.drop('날짜', axis=1)
test_df = test_df.drop('날짜', axis=1)

In [77]:
# 전처리 후 데이터 확인
print("전처리 후 데이터 형태:")
print("Train 데이터:", train_df.shape)
print("Test 데이터:", test_df.shape)

전처리 후 데이터 형태:
Train 데이터: (301, 3)
Test 데이터: (301, 3)


In [78]:
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CPI     301 non-null    float64
 1   금리      301 non-null    float64
 2   환율      301 non-null    float64
dtypes: float64(3)
memory usage: 7.2 KB
None


In [79]:
#train_df["날짜"] = pd.to_datetime(train_df["날짜"]) 
#test_df["날짜"] = pd.to_datetime(test_df["날짜"]) 

In [80]:
# 데이터 컬럼 확인
print("\nTrain 데이터 컬럼:", train_df.columns.tolist())
print("Test 데이터 컬럼:", test_df.columns.tolist())


Train 데이터 컬럼: ['CPI', '금리', '환율']
Test 데이터 컬럼: ['CPI', '금리', '환율']


In [81]:
# 스케일링 (MinMaxScaler)
scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = train_df.drop('CPI', axis=1)
y = train_df['CPI']


In [82]:
# 피처 수 확인
n_features = X.shape[1]
print(f"피처 수: {n_features}")

피처 수: 2


In [83]:
# fit(): 학습 데이터 X의 **최솟값(min)**과 **최댓값(max)**을 계산
# transform(): 그 계산된 값을 이용해 X의 각 값을 0~1 사이의 값으로 정규화
# X는 정규화된 numpy array가 되어 모델 학습에 적합한 형태가 됩니다.
# scaler.transform(test_df):위에서 학습 데이터로 fit()한 최소/최대값을 그대로 사용하여, 테스트 데이터 test_df도 동일한 방식으로 정규화
# 즉, 학습 기준에 맞게 테스트 데이터를 변환
# 이 과정을 통해 학습/테스트 간 스케일의 일관성을 유지
# test_df에는 반드시 train_df와 동일한 컬럼 수, 순서, 형식이 있어야 합니다

X_scaled = scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y.values.reshape(-1, 1))

X_test = test_df.drop('CPI', axis=1)
test_data_scaled = scaler.transform(X_test)


In [84]:
# 특성 이름 저장 (SHAP 시각화에 사용)
feature_names = train_df.drop('CPI', axis=1).columns.tolist()

In [85]:
from sklearn.model_selection import train_test_split
# train, validation set 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [86]:
# 데이터 형태 확인
print(f"X_train 형태(reshape 전): {X_train.shape}")
print(f"X_val 형태(reshape 전): {X_val.shape}")
print(f"test_data_scaled 형태(reshape 전): {test_data_scaled.shape}")

X_train 형태(reshape 전): (240, 2)
X_val 형태(reshape 전): (61, 2)
test_data_scaled 형태(reshape 전): (301, 2)


In [88]:
# LSTM 입력을 위한 reshape (samples, time_steps, features)
X_train = X_train.values.reshape(X_train.shape[0], 1, n_features)
X_val = X_val.values.reshape(X_val.shape[0], 1, n_features)
test_data = test_data_scaled.reshape(test_data_scaled.shape[0], 1, n_features)

In [89]:
# reshape 후 형태 확인
print(f"X_train 형태(reshape 후): {X_train.shape}")
print(f"X_val 형태(reshape 후): {X_val.shape}")
print(f"test_data_scaled 형태(reshape 후): {test_data_scaled.shape}")

X_train 형태(reshape 후): (240, 1, 2)
X_val 형태(reshape 후): (61, 1, 2)
test_data_scaled 형태(reshape 후): (301, 2)


In [90]:
# 2. LSTM 모델 구축
model = Sequential()
model.add(LSTM(64, input_shape=(1, n_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))  # 연속값 예측 → linear activation

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])  # 회귀 문제에 적합

c:\test\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [91]:
# 모델 구조 확인
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 1, 64)          │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,601 (115.63 KB)

 Trainable params: 29,601 (115.63 KB)

 Non-trainable params: 0 (0.00 B)

In [92]:
# Early Stopping - verbose=1로 설정하여 진행상황 확인
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,          # 10번의 에폭 동안 개선이 없으면 학습 중단
    restore_best_weights=True,    #  가장 좋은 성능을 보였던 모델의 가중치(weights)를 복원
    verbose=1             # 중단 시 메시지 출력
)

In [93]:
# 3. 모델 훈련 - verbose=1로 설정하여 진행상황 확인
epochs = 100
batch_size = 32

history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1             # 훈련 진행상황 출력
)

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - loss: 8.0297 - mae: 2.4404 - val_loss: 6.5926 - val_mae: 2.2005
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 5.9373 - mae: 2.1006 - val_loss: 5.7170 - val_mae: 2.0024
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.4877 - mae: 1.9518 - val_loss: 4.7768 - val_mae: 1.8018
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4.4102 - mae: 1.7275 - val_loss: 3.8497 - val_mae: 1.5925
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.8311 - mae: 1.5576 - val_loss: 3.1092 - val_mae: 1.4292
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.0577 - mae: 1.3720 - val_loss: 2.5008 - val_mae: 1.2853
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2.5268 - mae: 1.2959 - val_loss: 2.0788 - val_mae: 1.1821
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.3020 - mae: 1.2183 - val_loss: 1.8533 - val_mae: 1.1060
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.8742 - mae: 

In [94]:
# 학습이 몇 번째 에폭에서 멈췄는지 확인
actual_epochs = len(history.history['loss'])
print(f"학습이 {actual_epochs}번째 에폭에서 완료되었습니다.")

학습이 22번째 에폭에서 완료되었습니다.


In [98]:
# 예측 (정규화된 결과)
y_pred_val_scaled = model.predict(X_val, verbose=0)

# 역정규화 (예측값과 실제값)
y_pred_val = y_scaler.inverse_transform(y_pred_val_scaled)
y_val_actual = y_scaler.inverse_transform(y_val.values.reshape(-1, 1))

# 평가 지표 계산
mae = mean_absolute_error(y_val_actual, y_pred_val)
rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_val))
r2 = r2_score(y_val_actual, y_pred_val)

# 출력
print(f"📊 검증 데이터 성능 평가")
print(f"MAE  (평균 절대 오차): {mae:.4f}")
print(f"RMSE (평균 제곱근 오차): {rmse:.4f}")
print(f"R²   (결정 계수): {r2:.4f}")

📊 검증 데이터 성능 평가
MAE  (평균 절대 오차): 7.2047
RMSE (평균 제곱근 오차): 8.8637
R²   (결정 계수): -0.0000
